In [23]:
path_xlsx_parte = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [24]:
import sys
sys.path.append('../_amigocloud')

In [25]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [26]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [27]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [28]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry)'}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [29]:
df_lib = pd.DataFrame(data)

In [30]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,338,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
1,339,109,2024-08-22 00:56:52+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-21 19:29:52+00:00,2024-08-21 20:04:52+00:00,LIBERACION,32,18,66,CONCLUIDO,MT 02,None
2,25,112,None,AURELIO GARCIA,AURELIO GARCIA,2024-08-21 19:55:02+00:00,2024-08-21 20:32:02+00:00,LIBERACION,33,18,65,CONCLUIDO,MT 01,None
3,858,112,None,AURELIO GARCIA,AURELIO GARCIA,2024-08-21 19:55:02+00:00,2024-08-21 20:32:02+00:00,LIBERACION,33,18,65,CONCLUIDO,MT 01,None
4,214,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
5,216,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
6,217,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
7,854,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
8,856,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None
9,857,113,2024-08-22 13:56:58+00:00,EDUARDO REYES,EDUARDO REYES,2024-08-22 13:56:58+00:00,2024-08-23 14:35:58+00:00,LIBERACION,32,19,57,CONCLUIDO,MT 02,None


In [31]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[109, 110, 111, 112, 113, 114, 115, 116]

In [32]:
lista_id = list(set(df_lib['id']))
lista_id

[659,
 660,
 661,
 662,
 663,
 664,
 665,
 25,
 667,
 666,
 670,
 671,
 672,
 675,
 679,
 680,
 681,
 682,
 683,
 684,
 685,
 686,
 687,
 688,
 689,
 690,
 691,
 692,
 693,
 694,
 695,
 696,
 697,
 698,
 699,
 700,
 591,
 593,
 338,
 339,
 594,
 595,
 214,
 854,
 856,
 217,
 216,
 858,
 857,
 597]

In [33]:
nulos = df_lib.isnull().sum()
nulos

id                      0
idd                     0
fecha_registro          2
piloto_1                0
piloto_2                5
hora_inicio             0
hora_fin                0
labor                   0
temperatura             0
viento                  0
humedad                 0
estado_de_operacion     0
dron                    0
obs                    50
dtype: int64

In [13]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [14]:
if null_check:
    df_lib = None
else:
    None

In [15]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,443,89,2024-12-10 04:00:00+00:00,CHRISTIAN CALIZAYA,ALEJANDRO SANCHEZ,2024-08-10 22:00:00+00:00,2024-08-11 00:00:00+00:00,LIBERACION,24,3,60,CONCLUIDO,DA 01,None
1,452,89,2024-12-10 04:00:00+00:00,CHRISTIAN CALIZAYA,ALEJANDRO SANCHEZ,2024-08-10 22:00:00+00:00,2024-08-11 00:00:00+00:00,LIBERACION,24,3,60,CONCLUIDO,DA 01,None
2,461,90,2024-08-12 04:00:00+00:00,ALEJANDRO SANCHEZ,AURELIO GARCIA,2024-08-12 22:30:00+00:00,2024-12-14 03:40:00+00:00,LIBERACION,18,2,65,CONCLUIDO,DA 01,None
3,470,89,2024-12-10 04:00:00+00:00,CHRISTIAN CALIZAYA,ALEJANDRO SANCHEZ,2024-08-10 22:00:00+00:00,2024-08-11 00:00:00+00:00,LIBERACION,24,3,60,CONCLUIDO,DA 01,None
4,488,90,2024-08-12 04:00:00+00:00,ALEJANDRO SANCHEZ,AURELIO GARCIA,2024-08-12 22:30:00+00:00,2024-12-14 03:40:00+00:00,LIBERACION,18,2,65,CONCLUIDO,DA 01,None
5,502,87,2024-08-10 04:00:00+00:00,ALEJANDRO SANCHEZ,CHRISTIAN CALIZAYA,2024-08-10 20:30:00+00:00,2024-08-13 21:10:00+00:00,LIBERACION,29,4,27,CONCLUIDO,DA 01,None
6,453,91,2024-08-12 04:00:00+00:00,AURELIO GARCIA,ALEJANDRO SANCHEZ,2024-08-12 22:30:00+00:00,2024-08-14 03:30:00+00:00,LIBERACION,18,2,65,CONCLUIDO,DA 03,None
7,462,89,2024-12-10 04:00:00+00:00,CHRISTIAN CALIZAYA,ALEJANDRO SANCHEZ,2024-08-10 22:00:00+00:00,2024-08-11 00:00:00+00:00,LIBERACION,24,3,60,CONCLUIDO,DA 01,None
8,471,89,2024-12-10 04:00:00+00:00,CHRISTIAN CALIZAYA,ALEJANDRO SANCHEZ,2024-08-10 22:00:00+00:00,2024-08-11 00:00:00+00:00,LIBERACION,24,3,60,CONCLUIDO,DA 01,None
9,489,91,2024-08-12 04:00:00+00:00,AURELIO GARCIA,ALEJANDRO SANCHEZ,2024-08-12 22:30:00+00:00,2024-08-14 03:30:00+00:00,LIBERACION,18,2,65,CONCLUIDO,DA 03,None


In [16]:
df_parte = pd.read_excel(path_xlsx_parte)

In [17]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD,REPORTE
0,641,2024-05-23,21.0,8,13171,SUAREZ JUSTINIANO DAVID,1958,NUEVA ESPERAZANZA - SUAREZ DAVID,L1,17.000000,...,DA 02,ALEJANDRO SANCHEZ,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
1,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L9.2,17.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
2,525,2024-06-27,26.0,66,1218,BALCAZAR ALCANTARA JOSE LOLY,510,LA BELLA & SANTA ROSA I & VILLA FLORES,L13,10.710000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,CREDITO 2024,NaN,NaN,NaN,SI
3,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C10.3,2.490000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
4,558,2024-06-27,26.0,124,1530,CITTCA,299,CITTCA,C12.2,0.830000,...,DA 01,EDUARDO REYES,AURELIO GARCIA,NaN,NaN,SOLICITUD,NaN,NaN,NaN,SI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,NaN,NaT,NaN,81,42239,CORDOVA OLGUIN LEONARDO,636,SAN JUAN--CORDOVA,L6.3,2.044085,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2023,506.0,52.0,NaN,NaN
519,NaN,NaT,NaN,81,42239,CORDOVA OLGUIN LEONARDO,637,SILES--CORDOVA,L1.1,13.294597,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,507.0,52.0,NaN,NaN
520,NaN,NaT,NaN,81,42239,CORDOVA OLGUIN LEONARDO,367,LA AURORA Y SAN JUAN--CORDOVA,L3,2.826115,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,508.0,52.0,NaN,NaN
521,NaN,NaT,NaN,81,42239,CORDOVA OLGUIN LEONARDO,636,SAN JUAN--CORDOVA,L3,5.844763,...,NaN,NaN,NaN,NaN,NaN,SIEMBRA 2024,509.0,52.0,NaN,NaN


In [18]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [19]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\Usuario\AppData\Local\Temp\ipykernel_33140\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [20]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [21]:
lista_idd

[85, 86, 87, 88, 89, 90, 91]

In [22]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

se proceso IDD: 85
se proceso IDD: 86
se proceso IDD: 87
se proceso IDD: 88
se proceso IDD: 89
se proceso IDD: 90
se proceso IDD: 91


In [23]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)

se proceso ID: 443
se proceso ID: 445
se proceso ID: 446
se proceso ID: 447
se proceso ID: 448
se proceso ID: 449
se proceso ID: 450
se proceso ID: 451
se proceso ID: 452
se proceso ID: 453
se proceso ID: 454
se proceso ID: 455
se proceso ID: 456
se proceso ID: 457
se proceso ID: 458
se proceso ID: 459
se proceso ID: 460
se proceso ID: 461
se proceso ID: 462
se proceso ID: 463
se proceso ID: 465
se proceso ID: 466
se proceso ID: 467
se proceso ID: 469
se proceso ID: 470
se proceso ID: 471
se proceso ID: 472
se proceso ID: 473
se proceso ID: 475
se proceso ID: 476
se proceso ID: 477
se proceso ID: 376
se proceso ID: 488
se proceso ID: 489
se proceso ID: 491
se proceso ID: 492
se proceso ID: 493
se proceso ID: 494
se proceso ID: 378
se proceso ID: 495
se proceso ID: 498
se proceso ID: 499
se proceso ID: 500
se proceso ID: 501
se proceso ID: 502
se proceso ID: 503
se proceso ID: 370
se proceso ID: 371
se proceso ID: 373
se proceso ID: 379
se proceso ID: 380
se proceso ID: 381
se proceso I

## Copiar ids de PARTE DIARIO a PLANIFICACION

In [24]:
path_planificacion = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2024\SHP\PLANIFICACION.shp'
path_xlsx = r'G:\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [25]:
gdf_plan = gpd.read_file(path_planificacion)
df_xlsx = pd.read_excel(path_xlsx)

In [26]:
# Recorrer cada registro de df_xlsx
for index, row in df_xlsx.iterrows():
    id_value = row['ID']
    idd_value = row['IDD']
    
    # Actualizar el campo 'idd' en gdf_plan donde coincida el campo 'id'
    gdf_plan.loc[gdf_plan['id'] == id_value, 'idd'] = idd_value

In [27]:
gdf_plan.to_file(path_planificacion, driver='ESRI Shapefile')